In [ ]:
import numpy as np
import cv2

class GuidedFilter:

    def __init__(self, I, radius, epsilon):

        self._radius = 2 * radius + 1
        self._epsilon = epsilon
        self._I = self._toFloatImg(I)
        self._initFilter()

    def _toFloatImg(self, img):
        if img.dtype == np.float32:
            return img
        return ( 1.0 / 255.0 ) * np.float32(img)

    def _initFilter(self):
        I = self._I
        r = self._radius
        eps = self._epsilon

        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]


        self._Ir_mean = cv2.blur(Ir, (r, r))
        self._Ig_mean = cv2.blur(Ig, (r, r))
        self._Imean_b = cv2.blur(Ib, (r, r))

        Irr= cv2.blur(Ir ** 2, (r, r)) - self._Ir_mean ** 2 + eps                                       
        Irg = cv2.blur(Ir * Ig, (r, r)) - self._Ir_mean * self._Ig_mean                                  
        Irb= cv2.blur(Ir * Ib, (r, r)) - self._Ir_mean * self._Imean_b                                  
        Igg = cv2.blur(Ig * Ig, (r, r)) - self._Ig_mean * self._Ig_mean + eps                            
        Igb = cv2.blur(Ig * Ib, (r, r)) - self._Ig_mean * self._Imean_b                                  
        Ibb = cv2.blur(Ib * Ib, (r, r)) - self._Imean_b * self._Imean_b + eps                                                       


        inv_irr = Igg * Ibb -Igb *Igb                                                      
        inv_irg =Igb * Irb- Irg * Ibb                                                      
        inv_irb = Irg *Igb - Igg * Irb                                                     
        inv_igg = Irr* Ibb - Irb* Irb                                                     
        inv_igb = Irb* Irg - Irr*Igb                                                      
        inv_ibb = Irr* Igg - Irg * Irg                                                      
        
        cov_inv = inv_irr * Irr+ inv_irg * Irg + inv_irb * Irb                                   
        inv_irr /= cov_inv                                                                                     
        inv_irg /= cov_inv                                                                                     
        inv_irb /= cov_inv                                                                                     
        inv_igg /= cov_inv                                                                                     
        inv_igb /= cov_inv                                                                                     
        inv_ibb /= cov_inv                                                                                     
        
        self._inv_irr = inv_irr                                                                              
        self._inv_irg = inv_irg                                                                              
        self._inv_irb = inv_irb                                                                              
        self._inv_igg = inv_igg                                                                              
        self._inv_igb = inv_igb                                                                              
        self._inv_ibb = inv_ibb                  

    def _compcoeff(self, p):
        r = self._radius                                                             
        I = self._I                                                                 
        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]                                                          
        

        p_mean = cv2.blur(p, (r, r))                             
        Ipr_mean = cv2.blur(Ir * p, (r, r))                                                         
        Ipg_mean = cv2.blur(Ig * p, (r, r))                                                    
        Ipmean_b = cv2.blur(Ib * p, (r, r))             



        Ipr_cov = Ipr_mean - self._Ir_mean * p_mean                                                 
        Ipg_cov = Ipg_mean - self._Ig_mean * p_mean                                                     
        Ipb_cov = Ipmean_b - self._Imean_b * p_mean                                                       
                                                                                                                 
        ar = self._inv_irr * Ipr_cov + self._inv_irg * Ipg_cov + self._inv_irb * Ipb_cov                 
        ag = self._inv_irg * Ipr_cov + self._inv_igg * Ipg_cov + self._inv_igb * Ipb_cov                
        ab = self._inv_irb * Ipr_cov + self._inv_igb * Ipg_cov + self._inv_ibb * Ipb_cov    

        b = p_mean - ar * self._Ir_mean - ag * self._Ig_mean - ab * self._Imean_b                                                                                                                                         

        mean_ar = cv2.blur(ar, (r, r))          
        mean_ag = cv2.blur(ag, (r, r))                                                                   
        mean_ab = cv2.blur(ab, (r, r))                                                                      
        mean_b = cv2.blur(b, (r, r))                                                                                                                                              

        return mean_ar, mean_ag, mean_ab, mean_b            

    def _compout(self, ab, I):
    
        mean_ar, mean_ag, mean_ab, mean_b = ab
        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]
        q = mean_ar * Ir + mean_ag * Ig + mean_ab * Ib + mean_b
        return q

    def filter(self, p):

        p32f = self._toFloatImg(p)

        ab = self._compcoeff(p)
        return self._compout(ab, self._I)

